## 参考にしたページ

http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/

http://qiita.com/ysekky/items/c81ff24da0390a74fc6c

https://qiita.com/takechanman/items/6d1f65f94f7aaa016377

In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import math

# numpyのループを早くしてくれるやつ
import numba

## MFの部分

In [56]:
# 各セルの誤差を取得
@numba.jit
def get_rating_error(r, p, q):
    return r - np.dot(p, q)

# 行列全体の誤差を取得
@numba.jit
def get_error(R, P, Q, beta):
    error = 0.0
    for i in xrange(len(R)):
        for j in xrange(len(R[i])):
            # 評価値が0の場合は値が無いものとみなす
            if R[i][j] == 0:
                continue
            # ここは二乗誤差
            error += pow(get_rating_error(R[i][j], P[:,i], Q[:,j]), 2)
            # 正則化のための項目
            error += beta/2.0 * (np.linalg.norm(P) + np.linalg.norm(Q))
    return error

@numba.jit
def matrix_factorization(R, K, steps=5000, alpha=0.0002, beta=0.02, threshold=0.001):
    # 分解結果となる行列の初期値はランダムに決める
    P = np.random.rand(K, len(R))
    Q = np.random.rand(K, len(R[0]))
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                # 評価値が0の場合は値が無いものとみなす
                if(R[i][j] == 0 or math.isnan(R[i][j])):
                    continue
                err = get_rating_error(R[i][j], P[:, i], Q[:, j])
                # セルの内容を更新
                for k in xrange(K):
                    # 正則化も入れる
                    P[k][i] += alpha * (2 * err * Q[k][j] - beta * P[k][i])
                    Q[k][j] += alpha * (2 * err * P[k][i] - beta * Q[k][j])
        # 行列全体の誤差を計算
        error = get_error(R, P, Q, beta)
        # 誤差が閾値を超えたら終了
        if error < threshold:
            break
    return P, Q

## 小さいデータのサンプル

In [61]:
# np.NANの部分は未知
R = np.array([
        [5, 3, np.NAN, 1],
        [4, np.NAN, np.NAN, 1],
        [1, 1, np.NAN, 5],
        [1, np.NAN, np.NAN, 4],
        [np.NAN, 1, 5, 4],
        ]
    )

In [62]:
pd.DataFrame(R)

,0,1,2,3
0,5.0,3.0,NaN,1.0
1,4.0,NaN,NaN,1.0
2,1.0,1.0,NaN,5.0
3,1.0,NaN,NaN,4.0
4,NaN,1.0,5.0,4.0


In [63]:
# matrix_factorizationの計算
nP, nQ = matrix_factorization(R, 2, steps=5000)

In [64]:
# 分解後のアイテム行列とユーザ行列で近似した元の行列
# 欠損値が埋まっている！！！
nR = np.dot(nP.T, nQ)
pd.DataFrame(nR)

,0,1,2,3
0,5.003617,2.905015,5.034244,0.996812
1,3.957049,2.302793,4.140647,0.995599
2,1.097275,0.760517,4.749585,4.959958
3,0.951543,0.650959,3.866262,3.972794
4,2.081513,1.302945,4.883397,4.042160


## データの読み込み

In [2]:
file_path = './train/train_A.tsv'
sep = '\t'
target_df = pd.read_csv(file_path, sep)

In [53]:
ts = target_df[(target_df['event_type']==2)]['product_id']=='00001418_a'


4470       False
4736       False
8375       False
8472       False
8498       False
8499       False
8530       False
8538       False
8548       False
8549       False
8554       False
8564       False
8567       False
8577       False
8586       False
8597       False
8614       False
8617       False
8626       False
8655       False
8659       False
8708       False
8709       False
8730       False
8742       False
8760       False
8793       False
8820       False
8821       False
8822       False
           ...  
3372054    False
3372301    False
3372303    False
3372488    False
3372612    False
3373102    False
3373105    False
3373109    False
3373114    False
3373377    False
3373379    False
3373382    False
3373389    False
3373566    False
3373632    False
3373636    False
3373656    False
3373657    False
3373670    False
3373677    False
3373678    False
3373776    False
3373778    False
3373957    False
3373991    False
3374331    False
3374346    False
3374674    Fal

In [38]:
((target_df['user_id'].isin(target['user_id'])) & (target_df['product_id'].isin(target['product_id']))).value_counts()

False    2745165
True      631328
dtype: int64

In [16]:
target = target_df[target_df['event_type']==2]
target_df[(target_df['user_id'].isin(target['user_id'])) & (target_df['product_id'].isin(target['product_id']))].sort_values(by=['user_id','product_id','time_stamp']) 

,user_id,product_id,event_type,ad,time_stamp
678161,0000002_A,00001418_a,1,-1,2017-04-05 13:57:00.166
678193,0000002_A,00001653_a,1,-1,2017-04-05 14:23:03.897
678194,0000002_A,00001653_a,1,-1,2017-04-11 01:56:45.639
678187,0000002_A,00002183_a,1,-1,2017-04-05 14:01:31.197
678189,0000002_A,00002183_a,2,-1,2017-04-11 01:52:40.873
678188,0000002_A,00002183_a,1,-1,2017-04-11 01:52:43.391
678211,0000002_A,00002826_a,1,-1,2017-04-17 05:28:10.768
678210,0000002_A,00002826_a,1,-1,2017-04-17 05:28:14.929
678198,0000002_A,00003936_a,1,-1,2017-04-11 01:59:55.824
678197,0000002_A,00003936_a,1,-1,2017-04-11 02:00:43.467


In [4]:
# カラムを確認
target_df.columns

Index([u'user_id', u'product_id', u'event_type', u'ad', u'time_stamp'], dtype='object')

In [5]:
# コンバージョンは広告経由のみ評価対象とか書いてあってめんどくさい
# ad=1が広告経由のコンバージョン、ad=-1はコンバージョン以外のイベント
target_log_df = target_df[(target_df[u'ad'] == 1)|(target_df[u'ad'] == -1)]

In [6]:
# ピボットテーブル
# 値の無いところは0で埋める
# カートは評価に影響しない？ので0のままsumする
target_log_pivot_df  = pd.pivot_table(target_log_df, values=u'event_type', index=[u'user_id'], columns=[u'product_id'], aggfunc=np.sum)

In [ ]:
## 値入っているか心配であれば
# target_log_pivot_df.sum()

## 実行

In [25]:
import datetime
## 全部
# R = view_log_pivot_df.as_matrix()
##　時間がかかるので、、、一部
R = target_log_pivot_df.as_matrix()[:10]
print('start:{}'.format(datetime.datetime.now()))
nP, nQ = matrix_factorization(R, 1, steps=1)
print('end:{}'.format(datetime.datetime.now()))

start:2017-09-28 17:00:37.361868
end:2017-09-28 17:00:38.135771


In [26]:
# 分解後のユーザ行列
pd.DataFrame(nP)

,0,1,2,3,4,5,6,7,8,9
0,0.03622,0.358088,0.988548,0.919812,0.329793,0.064567,0.807213,0.059404,0.752638,0.090795


In [27]:
# 分解後のアイテム行列
pd.DataFrame(nQ)

,0,1,2,3,4,5,6,7,8,9,...,13854,13855,13856,13857,13858,13859,13860,13861,13862,13863
0,0.245574,0.61287,0.123506,0.638165,0.066832,0.811193,0.571889,0.523811,0.441569,0.436545,...,0.484044,0.309542,0.545908,0.295882,0.692991,0.368697,0.539915,0.81386,0.024282,0.755257


In [28]:
# 分解後のアイテム行列とユーザ行列で近似した元の行列
# 欠損値が埋まっている！！！
nR = np.dot(nP.T, nQ)
pd.DataFrame(nR)

,0,1,2,3,4,5,6,7,8,9,...,13854,13855,13856,13857,13858,13859,13860,13861,13862,13863
0,0.008895,0.022198,0.004473,0.023114,0.002421,0.029381,0.020714,0.018972,0.015994,0.015812,...,0.017532,0.011212,0.019773,0.010717,0.025100,0.013354,0.019556,0.029478,0.000879,0.027355
1,0.087937,0.219461,0.044226,0.228519,0.023932,0.290478,0.204786,0.187570,0.158120,0.156321,...,0.173330,0.110843,0.195483,0.105952,0.248152,0.132026,0.193337,0.291433,0.008695,0.270448
2,0.242762,0.605851,0.122091,0.630857,0.066067,0.801904,0.565340,0.517812,0.436513,0.431546,...,0.478501,0.305997,0.539657,0.292494,0.685056,0.364475,0.533732,0.804540,0.024004,0.746608
3,0.225882,0.563725,0.113602,0.586992,0.061473,0.746145,0.526030,0.481807,0.406161,0.401540,...,0.445229,0.284720,0.502133,0.272156,0.637422,0.339132,0.496620,0.748598,0.022335,0.694695
4,0.080989,0.202120,0.040731,0.210462,0.022041,0.267526,0.188605,0.172749,0.145626,0.143969,...,0.159634,0.102085,0.180037,0.097580,0.228544,0.121594,0.178060,0.268405,0.008008,0.249078
5,0.015856,0.039571,0.007974,0.041204,0.004315,0.052376,0.036925,0.033821,0.028511,0.028186,...,0.031253,0.019986,0.035248,0.019104,0.044744,0.023806,0.034861,0.052548,0.001568,0.048765
6,0.198231,0.494716,0.099695,0.515135,0.053948,0.654806,0.461636,0.422827,0.356440,0.352385,...,0.390726,0.249866,0.440664,0.238840,0.559392,0.297617,0.435826,0.656958,0.019600,0.609653
7,0.014588,0.036407,0.007337,0.037909,0.003970,0.048188,0.033972,0.031116,0.026231,0.025932,...,0.028754,0.018388,0.032429,0.017576,0.041166,0.021902,0.032073,0.048346,0.001442,0.044865
8,0.184828,0.461269,0.092955,0.480307,0.050300,0.610535,0.430425,0.394240,0.332342,0.328560,...,0.364310,0.232973,0.410871,0.222692,0.521572,0.277495,0.406360,0.612541,0.018275,0.568435
9,0.022297,0.055646,0.011214,0.057942,0.006068,0.073652,0.051925,0.047559,0.040092,0.039636,...,0.043949,0.028105,0.049566,0.026865,0.062920,0.033476,0.049022,0.073895,0.002205,0.068574


## レコメンド

In [ ]:
# 購買ログにフラグを立てる
buy_log_df = target_df
buy_log_df[u'buy_flg'] = target_df[u'event_type'].apply(lambda x: 1 if x == 3 else 0)
buy_log_pivot_df  = pd.pivot_table(buy_log_df, values=u'buy_flg', index=[u'user_id'], columns=[u'product_id'], aggfunc=np.sum, fill_value=0)

In [ ]:
# 嗜好情報をdfに
# 一部バージョン
ratio_df = pd.DataFrame(nR, index=view_log_pivot_df.index[:10], columns=view_log_pivot_df.columns)

In [ ]:
# レコメンド取得用関数
def get_recommend_items(user_id, ratio_df, buy_log_df, recommend_item_num=22):
    # product_idを対象ユーザのスコア順にソート
    products = ratio_df.loc[user_id].sort_values(ascending=False)
    # 購買履歴のある商品を省く
    buy_log = buy_log_df.loc[user_id]
    # レコメンド対象の商品リスト
    recommend_items = []
    for index in products.index:
        # 購買履歴のある商品は無視
        if buy_log[index] != 0:
            continue
        # 提出時の命名規則でappend
        recommend_items.append([user_id, index, len(recommend_items)])
        # 引数で指定した個数または規程個数(22個)を超えたら終了
        if len(recommend_items) >= recommend_item_num or len(recommend_items) >= 22:
            break
    return recommend_items

In [ ]:
get_recommend_items('0000000_A', ratio_df, buy_log_pivot_df)

## Scikit-learnを使ってNMFを計算する場合
通常のScikit-learnのNMFは欠損値を扱えないので、下記フォークのnmf_missingブランチの内容をインストールする。
`git checkout origin/nmf_missing`
https://github.com/TomDLT/scikit-learn

In [65]:
from sklearn.decomposition import NMF

# R = target_log_pivot_df.as_matrix()[:10]
R = np.array([
        [5, 3, np.NAN, 1],
        [4, np.NAN, np.NAN, 1],
        [1, 1, np.NAN, 5],
        [1, np.NAN, np.NAN, 4],
        [np.NAN, 1, 5, 4],
        ]
    )
k = 2
model = NMF(n_components=k, init='random', random_state=0, solver='mu', max_iter=1000)
P = model.fit_transform(R)
Q = model.components_
print("****************************")
print("k:",k)
print("Pは")
print(P)
print("Q^Tは")
print(Q)
print("P×Q^Tは")
print(np.dot(P,Q))
print("R-P×Q^Tは")
print(model.reconstruction_err_ )

****************************
('k:', 2)
Pは
[[ 0.25533737  2.31920431]
 [ 0.2749493   1.8405892 ]
 [ 1.74751251  0.10992921]
 [ 1.39405084  0.18346339]
 [ 1.39101969  0.25440826]]
Q^Tは
[[ 0.43984129  0.49350833  3.05253861  2.85396988]
 [ 2.10749104  1.23916027  2.96318486  0.11698132]]
P×Q^Tは
[[ 5.00001021  2.99987694  7.65165828  1.00002874]
 [ 3.99995931  2.41647477  6.29329943  1.00001158]
 [ 1.00030298  0.99863188  5.66008999  5.00020773]
 [ 0.99980858  0.91531625  4.79902998  4.00004091]
 [ 1.14799101  1.0017324   5.          3.9996893 ]]
R-P×Q^Tは
0.00227169772504


## 発展？
http://tech-blog.fancs.com/entry/factorization-machines